## Comparando os mapas Inventário x MapBiomas

## Importar os pacotes que serão usados

In [6]:
from osgeo import gdal, osr
import psutil
import sys
import numpy as np
import rasterio
from sklearn.metrics import confusion_matrix
from rasterio.plot import show
import pandas as pd
from numpy import savetxt

In [7]:
# # Carrega os arquivos
# dataset1 = '../dados/inventario_v4_2016/inventario_biomas_rasters/inventario_amazonia_byte.tif'
# dataset2 = '../dados/mapbiomas_v7_2016/biomas/mapbiomas_amazonia_byte.tif'

# with rasterio.open(dataset1) as src:
#     #show(src)
#     mapa1 = src.read()

# with rasterio.open(dataset2) as src:
#     #show(src)
#     mapa2 = src.read()

# # Convertendo os mapas em um array de uma só linha    
# mapa1 = mapa1.reshape(-1)
# mapa2 = mapa2.reshape(-1)

# print(len(mapa1))

# # Tabulação cruzada
# # tabulacao_cruzada = confusion_matrix(mapa1, mapa2)
# # print(tabulacao_cruzada)

# mapa1 = None
# mapa2 = None

## Fazer as checagens entre os mapas

Conferir o número de linhas, colunas, a projeção e a resolução de ambos os mapas e verificar se são coincidentes.

In [8]:
# Carrega os arquivos
dataset1 = gdal.Open('../dados/inventario_v4_2016/inventario_biomas_rasters/inventario_amazonia_byte.tif')
dataset2 = gdal.Open('../dados/mapbiomas_v7_2016/biomas/mapbiomas_amazonia_byte.tif')

#dataset1.SetProjection("EPSG:4674")

# verifica se as dimensões dos arquivos são iguais
if dataset1.RasterXSize != dataset2.RasterXSize or dataset1.RasterYSize != dataset2.RasterYSize:
    print("ERRO: As dimensões dos arquivos são diferentes.")
    sys.exit(1)

# verifica se as projeções dos arquivos são iguais
if osr.SpatialReference(dataset1.GetProjection()).GetAttrValue('AUTHORITY', 1) != osr.SpatialReference(dataset2.GetProjection()).GetAttrValue('AUTHORITY', 1):
    print("ERRO: As projeções dos arquivos são diferentes.")
    sys.exit(1)

# verifica se as resoluções dos arquivos são iguais
if round(dataset1.GetGeoTransform()[1], 7) != round(dataset1.GetGeoTransform()[1], 7):
    print("ERRO: As resoluções dos arquivos são diferentes.")
    sys.exit(1)

# se tudo estiver correto, exibe uma mensagem de confirmação
print("Os arquivos têm as mesmas dimensões, projeção e resolução.")

Os arquivos têm as mesmas dimensões, projeção e resolução.


## Checagem de memória disponível

Caso os mapas satifaçam as checagens, será feita uma verificação de memória disponível que será alocada para o processo de comparação dos mapas.

In [9]:
# Verifica se é possível obter a quantidade de memória disponível em tempo de execução
try:
    available_memory = psutil.virtual_memory().available

    # Define o tamanho máximo do bloco em bytes
    max_block_size_bytes = available_memory * 0.2  # 80% da memória disponível
    
except:
    # Caso contrário, define uma quantidade fixa de memória disponível
    available_memory = 1024 * 1024 * 1024  # 1 GB em bytes

    max_block_size_bytes = available_memory

print("Memória disponível:", available_memory, "\n")
print("Memória alocada:", max_block_size_bytes, "\n")

# Calcula o tamanho máximo do bloco em pixels
side_block = int((max_block_size_bytes / 2) ** 0.5)  # dividido por 2, pois são dois mapas

print("Lateral do bloco:", side_block)

Memória disponível: 6080663552 

Memória alocada: 1216132710.4 

Lateral do bloco: 24659


## Comparação dos mapas em pequenos batches

In [10]:
# obtém as dimensões dos arquivos
cols = dataset1.RasterXSize
rows = dataset1.RasterYSize
projection = dataset1.GetProjection()
geotransform = dataset1.GetGeoTransform()

print("Linhas, colunas e tamanho total:")
print(cols, rows, cols*rows, "\n")

num_blocos = 7

print("Altura e Largura do bloco")
step_col = int(cols/num_blocos)
step_row = int(rows/num_blocos)

print(step_col, step_row, "\n")

lista_classes = {}

for i in range(0, rows, step_row):
    for j in range(0, cols, step_col):

        # calcula o tamanho do bloco para esta parte
        print(f"Bloco_{i}_{j}", i*j)
        width = min(step_col, cols - j)
        height = min(step_row, rows - i)

        # le os blocos com os tamanhos determinados acima
        block1 = dataset1.ReadAsArray(j, i, xsize=width, ysize=height)
        block1 = block1.reshape(-1)

        block2 = dataset2.ReadAsArray(j, i, xsize=width, ysize=height)
        block2 = block2.reshape(-1)

        # calcula os valores de pixel unicos de cada mapa
        classes1 = np.unique(block1)
        classes2 = np.unique(block2)

        # concatena os valores e gera a lista de classes
        classes = np.unique(np.concatenate((classes1, classes2)))

        lista_classes[f"{i}_{j}"] = list(classes)

        #print("Iniciando matriz de confusao")
        block_confusion_matrix = confusion_matrix(block1, block2, labels=classes)
        print("Salvando tabulação cruzada referente ao bloco de altura", i, "e largura", j, "\n")

        # save to csv file
        savetxt(f'../confusion_matrix/amazonia/amazonia_{i}_{j}.csv', block_confusion_matrix, delimiter=',')

        #print(classes1, "\n", classes2, "\n")
        #print(classes)

Linhas, colunas e tamanho total:
112808 80069 9032423752 

Altura e Largura do bloco
16115 11438 

Bloco_0_0 0
Salvando tabulação cruzada referente ao bloco de altura 0 e largura 0 

Bloco_0_16115 0
Salvando tabulação cruzada referente ao bloco de altura 0 e largura 16115 

Bloco_0_32230 0
Salvando tabulação cruzada referente ao bloco de altura 0 e largura 32230 

Bloco_0_48345 0
Salvando tabulação cruzada referente ao bloco de altura 0 e largura 48345 

Bloco_0_64460 0
Salvando tabulação cruzada referente ao bloco de altura 0 e largura 64460 

Bloco_0_80575 0
Salvando tabulação cruzada referente ao bloco de altura 0 e largura 80575 

Bloco_0_96690 0
Salvando tabulação cruzada referente ao bloco de altura 0 e largura 96690 

Bloco_0_112805 0
Salvando tabulação cruzada referente ao bloco de altura 0 e largura 112805 

Bloco_11438_0 0
Salvando tabulação cruzada referente ao bloco de altura 11438 e largura 0 

Bloco_11438_16115 184323370
Salvando tabulação cruzada referente ao bloco de al

In [11]:
import json
import numpy as np
from datetime import date, datetime, timedelta

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.bool_):
            return bool(obj)
        if isinstance(obj, (np.floating, np.complexfloating)):
            return float(obj)
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, np.string_):
            return str(obj)
        if isinstance(obj, (datetime, date)):
            return obj.isoformat()
        if isinstance(obj, timedelta):
            return str(obj)
        return super(NpEncoder, self).default(obj)

In [12]:
import json
with open('../confusion_matrix/amazonia/lista_classes.txt', 'w') as fout:
    fout.write(json.dumps(lista_classes, cls=NpEncoder, indent=2))